In [6]:
# CODE REFERENCES
# https://towardsdatascience.com/how-to-build-an-image-classifier-for-waste-sorting-6d11d3c9c478
# https://github.com/danny95333/Trash-Classification-based-on-CNN
from linear_classifier import *
from linear_svm import *
from softmax import *
import time
import random
import shutil
import re
import os
from pathlib import Path
from PIL import Image
import numpy as np
import imghdr


In [7]:
# Declare the relative path to the PROCESSED trashnet data
# i.e. this path should go to "train", "test", "val" folders each of which 
# contain images for the train, test, and validation sets respectively.
dataRoot = "../../datasets/trashnet/data"

In [8]:
##########################
# DATA AUGMENTATION
# Loading the dataset into memory and applying transforms
# NOTE: May have to not use this for the baseline
##########################

# import torch
# from torchvision import transforms, datasets


# # Data augmentation and normalization for training
# # Just normalization for validation
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }


# # Create an ImageFolder dataloader for the input data
# # See https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder
# image_datasets = {x: datasets.ImageFolder(os.path.join(outDataPath, x),
#                                           data_transforms[x]) for x in ["train", "val"]}

# # Create DataLoader objects for each of the image datasets returned by ImageFolder
# # See https://pytorch.org/docs/stable/data.html
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
#                                               shuffle=True, num_workers=4) for x in ['train', 'val']}


# datasets_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

# class_names = image_datasets['train'].classes

# print(datasets_sizes)
# print(class_names)
# print(image_datasets)

In [32]:
# TODO: Implement data loading manually as np arrays
##########################
# Loading the dataset into memory and applying transforms
##########################

# TODO: Preinitialize X etc. as zeros

def load_image( infilename ) :
    '''
    Helper function to manually load in images. Accepts image path infilename 
    and returns the numpy array representation of the image data
    (Need final X to be of dims (N, H, W, D))
    '''
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

# Load the training data
arrayList = []
labelList = []
trainRoot = os.path.join(dataRoot, "train")
for c in os.listdir(os.path.join(trainRoot)):
    for item in os.listdir(os.path.join(trainRoot, c)):
        if imghdr.what(os.path.join(trainRoot, c, item)):
            # Convert the image to a numpy array
            data = load_image(os.path.join(trainRoot, c, item))
            # Add to the array which we will stack at the end to form X
            arrayList.append(data)
            # Add label to array for Y
            labelList.append(c)
X_train = np.stack(tuple(arrayList))
y_train = np.array(labelList)
        
            
# Load the validation data
arrayList = []
labelList = []
valRoot = os.path.join(dataRoot, "val")
for c in os.listdir(os.path.join(valRoot)):
    for item in os.listdir(os.path.join(valRoot, c)):
        if imghdr.what(os.path.join(valRoot, c, item)):
            # Convert the image to a numpy array
            data = load_image(os.path.join(valRoot, c, item))
            # Add to the array which we will stack at the end to form X
            arrayList.append(data)
            # Add label to array for Y
            labelList.append(c)
X_val = np.stack(tuple(arrayList))
y_val = np.array(labelList)

# Load the test data
arrayList = []
labelList = []
testRoot = os.path.join(dataRoot, "test")
for item in os.listdir(testRoot):
    if imghdr.what(os.path.join(testRoot, item)):
        # Produce label from filename
        ########
        # TODO: Go back to dataloader and change it so we bucket test data into the same 
        # subfolder structure as train and val data
        ########
        c = os.path.splitext(re.sub("[0-9]+", "", item))[0]
        data = load_image(os.path.join(testRoot, item))
        arrayList.append(data)
        labelList.append(c)
X_test = np.stack(tuple(arrayList))
y_test = np.array(labelList)


In [36]:
# Flatten X matrices down for SVM classification
imageDim = 384*512*3

X_train = X_train.reshape((X_train.shape[0], imageDim))
X_val = X_val.reshape((X_val.shape[0], imageDim))
X_test = X_test.reshape((X_test.shape[0], imageDim))

In [37]:
# Sanity check dimensions
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(1549, 589824)
(1549,)
(950, 589824)
(950,)
(970, 589824)
(970,)


In [40]:
# Encode the class labels
from sklearn.preprocessing import LabelEncoder
classes = ["cardboard", "compost", "glass", "metal", "paper", "plastic", "trash"]
le = LabelEncoder()
le.fit(classes)
y_train = le.transform(y_train)
y_val = le.transform(y_val)
y_test = le.transform(y_test)

In [41]:
# TRAIN THE MODEL
svm = LinearSVM()
tic = time.time()
print("Training model with {} rows of training data".format(X_train.shape[0]))
loss_hist = svm.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4,
                      num_iters=1500, verbose=True)
toc = time.time()
print('Training Complete. That took %fs' % (toc - tic))

Training model with 1549 rows of training data
iteration 0 / 1500: loss 51980.603545
iteration 100 / 1500: loss 20442.439398
iteration 200 / 1500: loss 7677.857989
iteration 300 / 1500: loss 3449.274737
iteration 400 / 1500: loss 2430.900509
iteration 500 / 1500: loss 2050.436010
iteration 600 / 1500: loss 1153.225606
iteration 700 / 1500: loss 1325.494756
iteration 800 / 1500: loss 1605.925831
iteration 900 / 1500: loss 1251.200552
iteration 1000 / 1500: loss 1318.792032
iteration 1100 / 1500: loss 973.098466
iteration 1200 / 1500: loss 896.706693
iteration 1300 / 1500: loss 692.072233
iteration 1400 / 1500: loss 1300.415057
Training Complete. That took 3829.853825s


In [42]:
# Evaluate the best svm on test set
y_test_pred = svm.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('linear SVM on raw pixels final test set accuracy: %f' % test_accuracy)

linear SVM on raw pixels final test set accuracy: 0.241237
